In [30]:
from pathlib import Path
from itertools import combinations
import numpy as np
import pandas as pd

PROJECT_ROOT = Path(r"C:\Users\lenovo\Desktop\scVOTE")
RESULTS_DIR  = PROJECT_ROOT / "results"
GENE_TOPIC_DIR = RESULTS_DIR / "gene_topic"

In [31]:
_gene_to_paths_cache = None
_total_pathways_cache = None

def load_gene_to_pathways(path: Path = PATHWAY_GENE_CSV):
    """
    读取C2+C5 pathway-gene，用于Extrinsic TC.
    自动缓存，避免每次重复加载。
    """
    global _gene_to_paths_cache, _total_pathways_cache

    if _gene_to_paths_cache is not None:
        return _gene_to_paths_cache, _total_pathways_cache

    df = pd.read_csv(path)[["pathway_name", "gene_symbol"]].dropna().drop_duplicates()

    total_pathways = df["pathway_name"].nunique()
    gene_to_paths = (
        df.groupby("gene_symbol")["pathway_name"]
          .apply(lambda x: set(x.tolist()))
          .to_dict()
    )

    _gene_to_paths_cache = gene_to_paths
    _total_pathways_cache = total_pathways

    print(f"[INFO] Loaded {total_pathways} pathways, {len(gene_to_paths)} genes with annotation")

    return gene_to_paths, total_pathways

In [ ]:
def compute_tc_extrinsic(
    gene_topic: np.ndarray,
    gene_names,
    top_n: int = 10,
    gene_to_paths=None,
    total_pathways=None,
    eps: float = 1e-12,
):
    """
    Extrinsic Topic Coherence based on MsigDB C2+C5 (NPMI)
    """
    gene_topic = np.asarray(gene_topic)
    n_topics, n_genes = gene_topic.shape
    gene_names = np.asarray(gene_names)

    assert len(gene_names) == n_genes, "gene_names长度必须与gene-topic的列一致"

    # Load pathway mapping (cached)
    if gene_to_paths is None:
        gene_to_paths, total_pathways = load_gene_to_pathways()

    tc_values = []

    for t in range(n_topics):
        weights = gene_topic[t, :]

        # Top-k gene index
        if top_n >= n_genes:
            top_idx = np.argsort(weights)[::-1]
        else:
            top_idx = np.argpartition(weights, -top_n)[-top_n:]
            top_idx = top_idx[np.argsort(weights[top_idx])[::-1]]

        top_genes = gene_names[top_idx]

        # Compute NPMI for every gene pair
        npmi_scores = []
        for g1, g2 in combinations(top_genes, 2):
            p1 = gene_to_paths.get(g1)
            p2 = gene_to_paths.get(g2)
            if not p1 or not p2:
                continue

            p_i = len(p1) / total_pathways
            p_j = len(p2) / total_pathways
            inter = p1 & p2
            if len(inter) == 0:
                continue

            p_ij = len(inter) / total_pathways
            if p_i <= eps or p_j <= eps or p_ij <= eps:
                continue

            pmi = np.log(p_ij / (p_i * p_j) + eps)
            npmi = pmi / (-np.log(p_ij + eps))
            npmi_scores.append(npmi)

        tc_values.append(np.mean(npmi_scores) if npmi_scores else np.nan)

    valid_tc = [x for x in tc_values if not np.isnan(x)]
    tc_mean = float(np.mean(valid_tc)) if len(valid_tc) > 0 else np.nan

    return tc_mean, tc_values

In [33]:
def compute_td(gene_topic: np.ndarray, top_n: int = 10):
    """
    Topic Diversity: fraction of unique genes in top-k sets across all topics.
    """
    gene_topic = np.asarray(gene_topic)
    n_topics, n_genes = gene_topic.shape

    all_top_idx = []

    for t in range(n_topics):
        weights = gene_topic[t, :]

        if top_n >= n_genes:
            idx = np.argsort(weights)[::-1]
        else:
            idx = np.argpartition(weights, -top_n)[-top_n:]
            idx = idx[np.argsort(weights[idx])[::-1]]

        all_top_idx.extend(idx.tolist())

    all_top_idx = np.asarray(all_top_idx)
    td_value = np.unique(all_top_idx).size / (n_topics * top_n)
    return float(td_value)

In [ ]:
def eval_one_gene_topic_file(csv_path, top_n=10):

    csv_path = Path(csv_path)
    fname = csv_path.name
    stem = csv_path.stem

    parts = stem.split("_")
    if len(parts) < 5 or parts[-2:] != ["gene", "topic"]:
        raise ValueError(f"文件名格式不符合要求: {fname}")

    model_name = parts[0]
    dataset_name = parts[1]
    K_value = parts[2]

    df = pd.read_csv(csv_path, index_col=0)

    gene_topic = df.values.T          #记得转置
    gene_names = df.index.values    

    gene_to_paths, total_pathways = load_gene_to_pathways()

    tc_mean, _ = compute_tc_extrinsic(
        gene_topic,
        gene_names,
        top_n=top_n,
        gene_to_paths=gene_to_paths,
        total_pathways=total_pathways
    )

    td_value = compute_td(gene_topic, top_n=top_n)

    return {
        "dataset": dataset_name,
        "model": model_name,
        "K": K_value,
        "file": fname,
        "TC": tc_mean,
        "TD": td_value,
    }

In [35]:
results = []

for csv in GENE_TOPIC_DIR.glob("*_gene_topic.csv"):
    res = eval_one_gene_topic_file(csv, top_n=10)
    results.append(res)

pd.DataFrame(results)

[INFO] Loaded 23789 pathways, 23590 genes with annotation


,dataset,model,K,file,TC,TD
0,kidney,LDA,K100,LDA_kidney_K100_gene_topic.csv,0.333824,0.374
1,kidney,LDA,K20,LDA_kidney_K20_gene_topic.csv,0.553118,0.075
2,kidney,LDA,K50,LDA_kidney_K50_gene_topic.csv,0.494112,0.178
3,lung,LDA,K100,LDA_lung_K100_gene_topic.csv,0.220159,0.572
4,lung,LDA,K20,LDA_lung_K20_gene_topic.csv,0.251217,0.100
5,lung,LDA,K50,LDA_lung_K50_gene_topic.csv,0.227363,0.272
6,PBMC4k,LDA,K100,LDA_PBMC4k_K100_gene_topic.csv,0.363324,0.168
7,PBMC4k,LDA,K20,LDA_PBMC4k_K20_gene_topic.csv,0.344361,0.565
8,PBMC4k,LDA,K50,LDA_PBMC4k_K50_gene_topic.csv,0.349224,0.382
9,Spleen,LDA,K100,LDA_Spleen_K100_gene_topic.csv,0.230059,0.900
